#### Import Libraries

In [2]:
import warnings
warnings.filterwarnings("ignore")
import nltk
import joblib
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from CustomTokenizer import CustomTokenizer
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.utils import resample, shuffle
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report, balanced_accuracy_score
pd.set_option('display.max_colwidth', 600)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ImportError: DLL load failed: The specified module could not be found.

#### Get Info from CSV

In [3]:
columns = ['Pregunta', 'Intencion']
df_train = shuffle(pd.read_csv('train.csv', usecols=columns, sep='|'))
df_test = shuffle(pd.read_csv('test_santander.csv', usecols=['id','Pregunta']))

NameError: name 'shuffle' is not defined

#### Helper Code

In [4]:
labelEncoders = dict()
def labelEncoder(df, column, params = {}):    
    le = preprocessing.LabelEncoder()        
    le.fit(df[column])    
    column_encoded = le.transform(df[column])
    labelEncoders[column] = le
    return column_encoded

In [5]:
df_train['Intencion_cat_label'] = df_train['Intencion'].str[4:]
df_train['Intencion_cat_label'] = df_train['Intencion_cat_label'].astype('int32')
df_train['Intencion_encoded'] = labelEncoder(df_train, 'Intencion')

NameError: name 'df_train' is not defined

In [6]:
df_categories = df_train.groupby(by='Intencion', sort=True, as_index=False).count()
df_categories.head(5)

NameError: name 'df_train' is not defined

In [7]:
#Frequency distribution of classes"
#train_outcome = pd.crosstab(index=df_train["Intencion"],  # Make a crosstab
#                              columns="count")      # Name the count column


sns.catplot(y='Intencion', x='Pregunta', kind="bar", data=df_categories, aspect=1, height=20)

NameError: name 'sns' is not defined

In [8]:
df_train.shape

NameError: name 'df_train' is not defined

In [9]:
X = df_train['Pregunta'].values
y = df_train['Intencion_encoded'].values

NameError: name 'df_train' is not defined

In [10]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
stopwords = nltk.corpus.stopwords.words('spanish')
tokenizer = CustomTokenizer()
tfidf_vect = TfidfVectorizer(lowercase=True, 
                             stop_words=stopwords, 
                             ngram_range = (1,2), 
                             strip_accents='ascii', 
                             tokenizer=tokenizer,
                             analyzer='word',
                             sublinear_tf=True,
                             token_pattern="[\w']+")

NameError: name 'nltk' is not defined

In [11]:
words = []
for index, row in df_train.iterrows():
    #print(row['Pregunta'])
    words.append(tokenizer.processAll(row['Pregunta']))    
df_train['words'] = words

NameError: name 'df_train' is not defined

In [12]:
df_train.sample(5)

NameError: name 'df_train' is not defined

In [13]:
X_train_vect = tfidf_vect.fit_transform(X)

NameError: name 'tfidf_vect' is not defined

In [14]:
X_train_vect.shape

NameError: name 'X_train_vect' is not defined

In [15]:
X_train_vect.toarray()

NameError: name 'X_train_vect' is not defined

In [16]:
len(tfidf_vect.get_feature_names())

NameError: name 'tfidf_vect' is not defined

In [17]:
tfidf_vect.get_feature_names()


NameError: name 'tfidf_vect' is not defined

In [18]:
cat_list = ['Cat_10', 'Cat_100', 'Cat_101', 'Cat_102', 'Cat_103', 'Cat_105', 'Cat_106', 'Cat_107', 'Cat_108', 'Cat_109']
df_train[df_train['Intencion'].isin(cat_list)].sort_values(by='Intencion')

NameError: name 'df_train' is not defined

In [19]:
import joblib
import pandas as pd 
import numpy as np
from CustomTokenizer import CustomTokenizer
from sklearn.utils import resample, shuffle
columns = ['Pregunta', 'Intencion']
tokenizer = CustomTokenizer()
df_train = shuffle(pd.read_csv('train.csv', usecols=columns, sep='|'))
samples = df_train.sample(5)
for sentence in samples['Pregunta'].values:
    print('after: ' + sentence)
    words = tokenizer.processAll(sentence)
    print('before: ' + str(words))

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "C:\ProgramData\Anaconda3\envs\data-science\python.exe"
  * The NumPy version is: "1.18.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: The specified module could not be found.


In [20]:
df_train.sample(10)

NameError: name 'df_train' is not defined

In [21]:
#import cProfile, pstats, io
#from pstats import SortKey
#pr = cProfile.Profile()
#pr.enable()

#words = []
#for index, row in df_train[:30].iterrows():
#    #print(row['Pregunta'])
#    words.append(tokenizer.processAll(row['Pregunta']))    
#df_train[:30]['words'] = words

#pr.disable()
#s = io.StringIO()
#sortby = SortKey.CUMULATIVE
#ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
#ps.print_stats()
#print(s.getvalue())

In [22]:
sentence =  "cuando me llega la tarjeta"

sentence = tokenizer.spell_correction_reserved_word_in_sentence(sentence)        

#Remove Numbers
sentence = tokenizer.removeNumbers(sentence)

#Remove Punctuation        
words = tokenizer.removePunc(sentence)

#Remove Stop Words                    
words = tokenizer.removeStopWords(words)

#spell_correction_reserved_word
words = [tokenizer.spell_correction_reserved_word(w) for w in words]

#spell_correction
words = [tokenizer.spell_correction(w) for w in words]

#Remove Special Characters              
words = [tokenizer.removeSpecialCharacters(w) for w in words]                      

#conjugate_verb
words = [tokenizer.conjugate_verb(w) for w in words]     

#Remove Accents                
words = [tokenizer.removeAccents(w) for w in words]

#Stem Words
words = [tokenizer.stemmer.stem(w) for w in words]          
print(words)

NameError: name 'tokenizer' is not defined

In [23]:
'necesito informacion sobre desarrollo urbanístico grand bourg de salta'

'necesito informacion sobre desarrollo urbanístico grand bourg de salta'